In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import torchvision.transforms as T
import torchvision.models as models
import numpy as np
from dataloader import Train
from network import Model
from helper import RandomTransWrapper, normalize_imu, normalize_speed, normalize_steering, normalize_image


In [2]:
model = torch.load('new_model.pt' , map_location=torch.device('cpu')) #<--- if current device is 'CPU'
model.eval() 
device = torch.device("cpu")
model.to(device)

Model(
  (mobile_net): MobileNetV2(
    (features): Sequential(
      (0): ConvNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(16, 96, kernel_size=(1, 1), str

# This part is to load an example batch from the dataset (not needed for use in Carla)

In [3]:
data_inf = Train(
            data_dir='./data_inf/',
            train_eval_flag="inf")

In [4]:
size_dataset_inf =data_inf.sequnece_len

inf_indices = list(range(size_dataset_inf))
inf_dataloader = DataLoader(data_inf,
                                  sampler=SubsetRandomSampler(inf_indices),
                                  batch_size=5, drop_last=True)

In [5]:
data, label = next(iter(inf_dataloader))

In [6]:
output = model(data)

In [7]:
output

tensor([[0.4918, 0.5004, 0.4880],
        [0.4918, 0.5004, 0.4880],
        [0.4918, 0.5004, 0.4880],
        [0.4918, 0.5004, 0.4880],
        [0.4918, 0.5004, 0.4880]], grad_fn=<SigmoidBackward0>)

# Create input

In [8]:
# Check if images need to be normalized or not
def create_input(img_color, img_depth_front, imu, speed, command):
    convert_tensor = T.ToTensor()
    img_rgb = img_color.astype(np.float32)
    img_rgb = convert_tensor(img_rgb)
    img_rgb= img_rgb.permute(1,2,0).unsqueeze_(0)

    
    img_depth = normalize_image(img_depth_front.astype(np.float32))
#     img_depth = (img_depth_front.astype(np.float32))
    img_depth = convert_tensor(img_depth)
    img_depth = img_depth.permute(1,2,0).unsqueeze_(0)

    imu = np.array(normalize_imu(imu))

    speed = (normalize_speed(speed))

    return [img_rgb,img_depth,torch.Tensor(imu).unsqueeze_(0),torch.Tensor(np.array([speed])),torch.Tensor(np.array([command]))]

In [9]:
# (n_samples, channels, height, width) # e.g., (1000, 1, 224, 224)


In [10]:
ex_rgb = data[0][0].numpy()
ex_depth = data[1][0].numpy()*255.0
ex_imu = data[2][0].numpy()
ex_speed = data[3][0].numpy()
ex_command = data[4][0].numpy()

# Create input from example data

In [11]:
inp = create_input(ex_rgb, ex_depth, ex_imu, ex_speed, ex_command)

# Make sure that RGB image is oriented ok

In [12]:
model(inp)

tensor([[0.4918, 0.5004, 0.4880]], grad_fn=<SigmoidBackward0>)

The first two inputs should be rounded to get binary data. 
If we need steering in degrees:

In [13]:
def norm_steer_to_degrees(norm_steer):
    return (norm_steer*180.0)-90.0